In [1]:
install.packages('ggplot2')
install.packages('caTools')
install.packages("readr")
install.packages("psych")
install.packages("fastDummies")


  There is a binary version available but the source version is later:
        binary source needs_compilation
ggplot2  3.4.4  3.5.1             FALSE



installing the source package 'ggplot2'





  There is a binary version available but the source version is later:
        binary source needs_compilation
caTools 1.18.2 1.18.3              TRUE



installing the source package 'caTools'





  There is a binary version available but the source version is later:
      binary source needs_compilation
readr  2.1.4  2.1.5              TRUE



installing the source package 'readr'





  There is a binary version available but the source version is later:
      binary   source needs_compilation
psych 2.3.12 2.4.6.26             FALSE



installing the source package 'psych'





  There is a binary version available but the source version is later:
            binary source needs_compilation
fastDummies  1.7.3  1.7.4             FALSE



installing the source package 'fastDummies'




In [2]:
library(ggplot2)
library(caTools)
library(readr)
library(psych)
library(dplyr)
library(fastDummies)



Attaching package: 'psych'


The following objects are masked from 'package:ggplot2':

    %+%, alpha



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




# 1. DATA ANALYSIS

### 1. Import the data set. Make sure the column names are imported as intended.

In [3]:
data <- read_csv("/Users/gabriel/Documents/GitHub/CausalAI-Course/Labs/Assignment/Assignment_1/wage2015_subsample_inference.csv")
print(data)
spec(data) 

Rows: 5150 Columns: 21
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (21): rownames, wage, lwage, sex, shs, hsg, scl, clg, ad, mw, so, we, ne...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 5,150 x 21
   rownames  wage lwage   sex   shs   hsg   scl   clg    ad    mw    so    we
      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1       10  9.62  2.26     1     0     0     0     1     0     0     0     0
 2       12 48.1   3.87     0     0     0     0     1     0     0     0     0
 3       15 11.1   2.40     0     0     1     0     0     0     0     0     0
 4       18 13.9   2.63     1     0     0     0     0     1     0     0     0
 5       19 28.8   3.36     1     0     0     0     1     0     0     0     0
 6       30 11.7   2.46     1     0     0     0     1     0     0     0     0
 7       43 19.2   2.96     1     0     1     0     0     0     0     0     0
 8       44 19.2   2.96     0     0     1     0     0     0     0     0     0
 9       47 12     2.48     1     0     1     0     0     0     0     0     0
10       71 19.2   2.96     1     0     0     0     1     0     0     0     0
# i 5,140 more rows
# i 9 more variables:

cols(
  rownames = col_double(),
  wage = col_double(),
  lwage = col_double(),
  sex = col_double(),
  shs = col_double(),
  hsg = col_double(),
  scl = col_double(),
  clg = col_double(),
  ad = col_double(),
  mw = col_double(),
  so = col_double(),
  we = col_double(),
  ne = col_double(),
  exp1 = col_double(),
  exp2 = col_double(),
  exp3 = col_double(),
  exp4 = col_double(),
  occ = col_double(),
  occ2 = col_double(),
  ind = col_double(),
  ind2 = col_double()
)

### 2. Are there missing values? Display the number of missings (if any) of each variable in the data set.

In [4]:
missing_values <- colSums(is.na(data))
missing_df <- data.frame(Variable = names(missing_values), Missings = missing_values, row.names = NULL)
print(missing_df)
# No hay missing values

   Variable Missings
1  rownames        0
2      wage        0
3     lwage        0
4       sex        0
5       shs        0
6       hsg        0
7       scl        0
8       clg        0
9        ad        0
10       mw        0
11       so        0
12       we        0
13       ne        0
14     exp1        0
15     exp2        0
16     exp3        0
17     exp4        0
18      occ        0
19     occ2        0
20      ind        0
21     ind2        0


### 3. Report descriptive statistics of the variables (mean, standard deviation, percentiles, etc.). Interpret your results.

In [5]:
summary_data <- sapply(data, summary)
summary_df <- as.data.frame(t(summary_data))
print(summary_df)
dim(data)

               Min.     1st Qu.       Median         Mean      3rd Qu.
rownames  10.000000 7132.500000 15260.000000 1.563635e+04 24337.750000
wage       3.021978   13.461538    19.230769 2.341041e+01    27.777778
lwage      1.105912    2.599837     2.956512 2.970787e+00     3.324236
sex        0.000000    0.000000     0.000000 4.444660e-01     1.000000
shs        0.000000    0.000000     0.000000 2.330097e-02     0.000000
hsg        0.000000    0.000000     0.000000 2.438835e-01     0.000000
scl        0.000000    0.000000     0.000000 2.780583e-01     1.000000
clg        0.000000    0.000000     0.000000 3.176699e-01     1.000000
ad         0.000000    0.000000     0.000000 1.370874e-01     0.000000
mw         0.000000    0.000000     0.000000 2.596117e-01     1.000000
so         0.000000    0.000000     0.000000 2.965049e-01     1.000000
we         0.000000    0.000000     0.000000 2.161165e-01     0.000000
ne         0.000000    0.000000     0.000000 2.277670e-01     0.000000
exp1  

[1] 5150   21

Observamos:

### 4. How many women with a college graduate degree (clg) or above have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [6]:
# Filtrar las mujeres con título universitario o superior cuyo salario está en el 25% superior
richest <- quantile(data$wage, 0.75)
women <- subset(data, sex == 1 & (clg == 1 | ad == 1))
women_richest <- subset(women, wage >= richest)
print(women_richest)

# Contar cuántas mujeres cumplen con las condiciones
num_women_richest <- nrow(women_richest)
cat('Number of women with a university degree or higher who belong to the richest 25%:', num_women_richest, '\n')

# A tibble: 419 x 21
   rownames  wage lwage   sex   shs   hsg   scl   clg    ad    mw    so    we
      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1       19  28.8  3.36     1     0     0     0     1     0     0     0     0
 2      191  42.3  3.74     1     0     0     0     0     1     0     0     0
 3      232  41.2  3.72     1     0     0     0     1     0     0     0     0
 4      319 100    4.61     1     0     0     0     0     1     0     0     0
 5      563  33.7  3.52     1     0     0     0     1     0     0     0     0
 6      842  36.1  3.59     1     0     0     0     1     0     0     0     0
 7      865  31.2  3.44     1     0     0     0     1     0     0     0     0
 8      892  28.8  3.36     1     0     0     0     1     0     0     0     0
 9     1052  48.1  3.87     1     0     0     0     1     0     0     0     0
10     1076  34.6  3.54     1     0     0     0     1     0     0     0     0
# i 409 more rows
# i 9 more variables: ne 

### 5. How many men with a high school graduate degree (hsg) or below have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [7]:
# Filtrar los hombres con "some highschool" o "highschool graduate" cuyo salario está en el 25% superior
richest <- quantile(data$wage, 0.75)
men <- subset(data, sex == 0 & (shs == 1 | hsg == 1))
men_richest <- subset(men, wage >= richest)
print(men_richest)

# Contar cuántos hombres cumplen con las condiciones
num_men_richest  <- nrow(men_richest)
cat('Number of men who graduated from high school or with a lower degree who belong to the richest 25%:', num_men_richest , '\n')

# A tibble: 118 x 21
   rownames  wage lwage   sex   shs   hsg   scl   clg    ad    mw    so    we
      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1      113  27.9  3.33     0     0     1     0     0     0     0     0     0
 2      276  28.8  3.36     0     0     1     0     0     0     0     0     0
 3      467  28.8  3.36     0     0     1     0     0     0     0     0     0
 4      858  28.8  3.36     0     0     1     0     0     0     0     0     0
 5      876  29.7  3.39     0     0     1     0     0     0     0     0     0
 6     1065  27.8  3.32     0     0     1     0     0     0     0     0     0
 7     1359  35.6  3.57     0     0     1     0     0     0     0     0     0
 8     1870  36.3  3.59     0     0     1     0     0     0     0     0     0
 9     2194  43.3  3.77     0     0     1     0     0     0     0     0     0
10     2214  43.3  3.77     0     0     1     0     0     0     0     0     0
# i 108 more rows
# i 9 more variables: ne 

### 6. Create two dataframes. One containing only the log(wage) and the other containig every variable of the data set but the wage related variables.

In [8]:
salario <- data['lwage']
sin_salario <- data[, !(colnames(data) %in% c('lwage', 'wage'))]
print(salario)
print(sin_salario)

# A tibble: 5,150 x 1
   lwage
   <dbl>
 1  2.26
 2  3.87
 3  2.40
 4  2.63
 5  3.36
 6  2.46
 7  2.96
 8  2.96
 9  2.48
10  2.96
# i 5,140 more rows
# A tibble: 5,150 x 19
   rownames   sex   shs   hsg   scl   clg    ad    mw    so    we    ne  exp1
      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1       10     1     0     0     0     1     0     0     0     0     1     7
 2       12     0     0     0     0     1     0     0     0     0     1    31
 3       15     0     0     1     0     0     0     0     0     0     1    18
 4       18     1     0     0     0     0     1     0     0     0     1    25
 5       19     1     0     0     0     1     0     0     0     0     1    22
 6       30     1     0     0     0     1     0     0     0     0     1     1
 7       43     1     0     1     0     0     0     0     0     0     1    42
 8       44     0     0     1     0     0     0     0     0     0     1    37
 9       47     1     0     1     0     0     0

# 2. DATA WRANGLING

### 7. Make an array for our  $Y$ variable, which will be the logarithm of wage ($lwage$ column) 

In [9]:
Y <- data$lwage
Y_matrix <- matrix(Y, ncol = 1)
cat("Matriz para la variable Y (logaritmo del salario):\n")
print(Y_matrix)

Matriz para la variable Y (logaritmo del salario):
            [,1]
   [1,] 2.263364
   [2,] 3.872802
   [3,] 2.403126
   [4,] 2.634928
   [5,] 3.361977
   [6,] 2.462215
   [7,] 2.956512
   [8,] 2.956512
   [9,] 2.484907
  [10,] 2.956512
  [11,] 2.851151
  [12,] 2.486508
  [13,] 2.486508
  [14,] 2.599837
  [15,] 2.793993
  [16,] 3.328075
  [17,] 3.072693
  [18,] 2.158004
  [19,] 2.956512
  [20,] 2.579217
  [21,] 2.368725
  [22,] 2.445686
  [23,] 2.878550
  [24,] 2.956512
  [25,] 2.791432
  [26,] 2.956512
  [27,] 2.668829
  [28,] 2.484907
  [29,] 2.822980
  [30,] 2.979111
  [31,] 3.744969
  [32,] 2.634928
  [33,] 2.312155
  [34,] 2.642854
  [35,] 2.482054
  [36,] 2.445686
  [37,] 3.179655
  [38,] 2.789457
  [39,] 2.445686
  [40,] 3.718652
  [41,] 3.074295
  [42,] 2.417515
  [43,] 1.906689
  [44,] 2.312155
  [45,] 2.486508
  [46,] 2.640659
  [47,] 3.361977
  [48,] 1.975682
  [49,] 2.466305
  [50,] 3.292984
  [51,] 2.560096
  [52,] 2.570849
  [53,] 2.733368
  [54,] 3.113515
  [55,] 4.6051

### 8. Make three new arrays for our predictors:

#### 8.1. The basic model will have the columns sex hsg scl clg ad so we ne exp1 occ2 ind2. Make sure to convert occ2 and ind2 to dummies and to drop the first dummy value to prevent multicolinearity. This means:

$sex + exp1 + hsg + scl + clg + ad + so + we + ne + dummy(occ2) + dummy(ind2)$ 

In [10]:
# Seleccionar las columnas deseadas
predictors <- sin_salario[, c('sex','exp1', 'hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne', 'occ2', 'ind2')]

# Convertir 'occ2' y 'ind2' en dummies (variables categóricas) sin incluir la primera categoría
predictors <- dummy_cols(predictors, select_columns = c('occ2', 'ind2'), remove_first_dummy = TRUE)

predictors <- predictors[, !colnames(predictors) %in% c('occ2', 'ind2')]

print(colnames(predictors))
print(dim(predictors))

 [1] "sex"     "exp1"    "hsg"     "scl"     "clg"     "ad"      "so"     
 [8] "we"      "ne"      "occ2_2"  "occ2_3"  "occ2_4"  "occ2_5"  "occ2_6" 
[15] "occ2_7"  "occ2_8"  "occ2_9"  "occ2_10" "occ2_11" "occ2_12" "occ2_13"
[22] "occ2_14" "occ2_15" "occ2_16" "occ2_17" "occ2_18" "occ2_19" "occ2_20"
[29] "occ2_21" "occ2_22" "ind2_3"  "ind2_4"  "ind2_5"  "ind2_6"  "ind2_7" 
[36] "ind2_8"  "ind2_9"  "ind2_10" "ind2_11" "ind2_12" "ind2_13" "ind2_14"
[43] "ind2_15" "ind2_16" "ind2_17" "ind2_18" "ind2_19" "ind2_20" "ind2_21"
[50] "ind2_22"
[1] 5150   50


In [11]:
# Convertir el dataframe a una matriz
predictors_matrix <- as.matrix(predictors)
print(predictors_matrix)
print(dim(predictors_matrix))

        sex exp1 hsg scl clg ad so we ne occ2_2 occ2_3 occ2_4 occ2_5 occ2_6
   [1,]   1  7.0   0   0   1  0  0  0  1      0      0      0      0      0
   [2,]   0 31.0   0   0   1  0  0  0  1      0      0      0      0      0
   [3,]   0 18.0   1   0   0  0  0  0  1      0      0      0      0      0
   [4,]   1 25.0   0   0   0  1  0  0  1      0      0      0      0      0
   [5,]   1 22.0   0   0   1  0  0  0  1      0      0      0      0      1
   [6,]   1  1.0   0   0   1  0  0  0  1      0      0      0      1      0
   [7,]   1 42.0   1   0   0  0  0  0  1      0      0      0      0      0
   [8,]   0 37.0   1   0   0  0  0  0  1      0      0      0      0      0
   [9,]   1 31.0   1   0   0  0  0  0  1      0      0      0      0      0
  [10,]   1  4.0   0   0   1  0  0  0  1      0      0      0      0      0
  [11,]   1  7.0   1   0   0  0  0  0  1      0      0      0      0      0
  [12,]   0 30.0   1   0   0  0  0  0  1      0      0      0      0      0
  [13,]   1 

### 8.2 The flexible model will have the same columns, and will also include polynomials for experience (exp2 exp3 exp4), as well as the interactions between all experience variables and other variables except for sex. This means:

$sex + exp1 + exp2 + exp3 + exp4 + hsg + scl + clg + ad + so + we + ne + dummy(occ2) + dummy(ind2) +(exp1 + exp2 + exp3 + exp4)$ X $(hsg + scl + clg + ad + so + we + ne + dummy(occ2) + dummy(ind2))$ 

Hint: you can use a for loop to multiply the desired variables and create the interactions. Some packages might also have ways to easily specify the model variables with strings like R does by default

In [12]:
# Replicamos lo mismo
predictors_2 <- sin_salario[, c('sex', 'exp1', 'exp2', 'exp3', 'exp4', 'hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne', 'occ2', 'ind2')]

# Convertir 'occ2' y 'ind2' en dummies
predictors_2 <- dummy_cols(predictors_2, select_columns = c('occ2', 'ind2'), remove_first_dummy = TRUE)
predictors_2 <- predictors_2[, !colnames(predictors_2) %in% c('occ2', 'ind2')] # es ncesario este paso, R no elimina las variables originales

# Columna con la suma de las variables de experiencia
predictors_2$exp <- rowSums(predictors_2[, c('exp1', 'exp2', 'exp3', 'exp4')])

# Columna con la suma de las demás variables excepto las dummies y 'sex'
predictors_2$suma <- rowSums(predictors_2[, c('hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne')])

# Crear una columna con la suma de las dummies
dummy_columns <- grep('^occ2_|^ind2_', colnames(predictors_2), value = TRUE)
predictors_2$dummy_sum <- rowSums(predictors_2[, dummy_columns])

# Crear una columna con la suma de la suma de dummies y otras variables
predictors_2$sum_2 <- predictors_2$suma + predictors_2$dummy_sum

# Crear una columna de interacción
predictors_2$interaction <- predictors_2$exp * predictors_2$sum_2

# Eliminar las columnas intermedias
predictors_2 <- predictors_2[, !(colnames(predictors_2) %in% c('exp', 'suma', 'dummy_sum', 'sum_2'))]

print(colnames(predictors_2))
# predictors_2
print(dim(predictors_2))

 [1] "sex"         "exp1"        "exp2"        "exp3"        "exp4"       
 [6] "hsg"         "scl"         "clg"         "ad"          "so"         
[11] "we"          "ne"          "occ2_2"      "occ2_3"      "occ2_4"     
[16] "occ2_5"      "occ2_6"      "occ2_7"      "occ2_8"      "occ2_9"     
[21] "occ2_10"     "occ2_11"     "occ2_12"     "occ2_13"     "occ2_14"    
[26] "occ2_15"     "occ2_16"     "occ2_17"     "occ2_18"     "occ2_19"    
[31] "occ2_20"     "occ2_21"     "occ2_22"     "ind2_3"      "ind2_4"     
[36] "ind2_5"      "ind2_6"      "ind2_7"      "ind2_8"      "ind2_9"     
[41] "ind2_10"     "ind2_11"     "ind2_12"     "ind2_13"     "ind2_14"    
[46] "ind2_15"     "ind2_16"     "ind2_17"     "ind2_18"     "ind2_19"    
[51] "ind2_20"     "ind2_21"     "ind2_22"     "interaction"
[1] 5150   54


In [13]:
# Convertir el dataframe a una matriz
predictors2_matrix <- as.matrix(predictors_2)
print(predictors2_matrix)
print(dim(predictors2_matrix))

        sex exp1    exp2       exp3         exp4 hsg scl clg ad so we ne occ2_2
   [1,]   1  7.0  0.4900   0.343000   0.24010000   0   0   1  0  0  0  1      0
   [2,]   0 31.0  9.6100  29.791000  92.35210000   0   0   1  0  0  0  1      0
   [3,]   0 18.0  3.2400   5.832000  10.49760000   1   0   0  0  0  0  1      0
   [4,]   1 25.0  6.2500  15.625000  39.06250000   0   0   0  1  0  0  1      0
   [5,]   1 22.0  4.8400  10.648000  23.42560000   0   0   1  0  0  0  1      0
   [6,]   1  1.0  0.0100   0.001000   0.00010000   0   0   1  0  0  0  1      0
   [7,]   1 42.0 17.6400  74.088000 311.16960000   1   0   0  0  0  0  1      0
   [8,]   0 37.0 13.6900  50.653000 187.41610000   1   0   0  0  0  0  1      0
   [9,]   1 31.0  9.6100  29.791000  92.35210000   1   0   0  0  0  0  1      0
  [10,]   1  4.0  0.1600   0.064000   0.02560000   0   0   1  0  0  0  1      0
  [11,]   1  7.0  0.4900   0.343000   0.24010000   1   0   0  0  0  0  1      0
  [12,]   0 30.0  9.0000  27.000000  81.

### 8.3 The extra-flexible model will include all two-way interactions between variables, except for sex. This means:

$sex + (exp1 + exp2 + exp3 + exp4 + hsg + scl + clg + ad + so + we + ne + dummy(occ2) + dummy(ind2))^2$ 

Hint: If you use a for loop here, you might create several variables identical to others. You can make another procedure to get rid of duplicates

In [14]:
predictors_3 <- sin_salario[, c('sex', 'exp1', 'exp2', 'exp3', 'exp4', 'hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne', 'occ2', 'ind2')]
predictors_3 <- dummy_cols(predictors_3, select_columns = c('occ2', 'ind2'), remove_first_dummy = TRUE)
predictors_3 <- predictors_3[, !colnames(predictors_3) %in% c('occ2', 'ind2')]

# Todas las columnas excepto 'sex'
columns_to_include <- setdiff(colnames(predictors_3), 'sex')

# Suma de todas las variables seleccionadas (excepto 'sex')
predictors_3$sum_vars <- rowSums(predictors_3[, columns_to_include])

# Nueva variable: cuadrado de la suma de todas las variables
predictors_3$twoway_interaction <- predictors_3$sum_vars^2

predictors_3 <- predictors_3[, !(colnames(predictors_3) %in% c('exp1', 'exp2', 'exp3', 'exp4', 'hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne', 'sum_vars'))]
predictors_3 <- predictors_3[, !grepl("^occ2_", colnames(predictors_3))]
predictors_3 <- predictors_3[, !grepl("^ind2_", colnames(predictors_3))]

# Mostrar el resultado
print(colnames(predictors_3))
print(dim(predictors_3))

[1] "sex"                "twoway_interaction"
[1] 5150    2


In [15]:
# Convertir el dataframe a una matriz
predictors3_matrix <- as.matrix(predictors_3)
print(predictors3_matrix)
print(dim(predictors3_matrix))

        sex twoway_interaction
   [1,]   1       1.457597e+02
   [2,]   0       2.780660e+04
   [3,]   0       1.728032e+03
   [4,]   1       7.909879e+03
   [5,]   1       4.213775e+03
   [6,]   1       2.511112e+01
   [7,]   1       2.015091e+05
   [8,]   0       8.570789e+04
   [9,]   1       2.780660e+04
  [10,]   1       6.805590e+01
  [11,]   1       1.457597e+02
  [12,]   0       2.280100e+04
  [13,]   1       1.012113e+02
  [14,]   1       3.022650e+03
  [15,]   1       7.909879e+03
  [16,]   0       1.102878e+03
  [17,]   0       1.234701e+04
  [18,]   0       5.898826e+01
  [19,]   0       1.146099e+02
  [20,]   0       1.839170e+02
  [21,]   1       1.294050e+02
  [22,]   1       2.061703e+02
  [23,]   1       3.611938e+02
  [24,]   0       3.611938e+02
  [25,]   1       7.096147e+03
  [26,]   0       1.839170e+02
  [27,]   1       7.119141e+01
  [28,]   0       7.805071e+01
  [29,]   0       1.839170e+02
  [30,]   0       1.000672e+04
  [31,]   1       1.600000e+01
  [32,] 

# 3. LINEAR REGRESSIONS

9. was omitted

### 10. Split each of the dataframes created (basic, flexible and extra-flexible models) into a training sample (80% of the data) and a test sample. Use the normalized data for this. Hint: You do not need to normalize the data for dummy variables.

In [16]:
# Formamos el conjunto de datos para cada modelo
data_1 <- data.frame(salario,predictors_matrix)
data_2 <- data.frame(salario,predictors2_matrix)
data_3 <- data.frame(salario,predictors3_matrix)
print(dim(data_1))
print(dim(data_2))
print(dim(data_3))

[1] 5150   51
[1] 5150   55
[1] 5150    3
